In [3]:
from pymongo import MongoClient
import psycopg2
import pickle
import re
import numpy as np
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim
geolocator = Nominatim()
import time
import sys
sys.path.insert(0, './text2num/')
import text2num
from dateutil import parser
import matplotlib.pyplot as plt
%matplotlib inline

###Load pickle file

In [2]:
with open("ufo_nosummary.pkl","r") as pf:
    d = pickle.load(pf)

###Make into pandas df

In [3]:
df = pd.DataFrame([d["city"],d["title"],d["shape"],d["duration"],d["state"],d["link"],d["date"]])
df = df.transpose()
df.columns = ['city','title','shape','duration','state','link','date']

###Shapes

In [192]:
def shapes(x):
    dic = {"triangular":"triangle","triange":"triangle","round":"circle",
           "changed":"changing","pyramid":"triangle","flare":"light",
           "dome":"disk","hexagon":"other","crescent":"other","delta":"chevron"}
    if x in dic.keys():
        return dic[x]
    else:
        return x

In [193]:
df['shape'] = df['shape'].map(lambda x: shapes(x.lower()) if x else "unknown")

###Date

In [7]:
def standardize_dt(dateStr):
    for fmt in ["%m/%d/%y","%m/%d/%y %H:%M"]:
        try:
            return datetime.strptime(dateStr, fmt)
        except ValueError:
            continue

In [8]:
#not using

def date_only(x):
    reg_date = "\d{1,2}\/\d{1,2}\/\d{2}"
    result = re.findall(reg_date,str(x))
    if len(result)>0:
        return datetime.strptime(result[0], "%m/%d/%y")
    else:
        return "None"
def time_only(x):
    reg_time = "\d{2}\:\d{2}"
    result = re.findall(reg_time,str(x))
    if len(result)>0:
        return datetime.strptime(result[0], "%H:%M")
    else:
        return "None"

In [8]:
df['date_datetime'] = df['date'].map(lambda x: standardize_dt(str(x)))

###Day of week

In [198]:
df['day']=df['date_datetime'].map(lambda x: x.weekday())

###Duration

In [98]:
def convertTime(elem):  
    t = 1
    lower = elem.lower()
    #TODO: learn regex
    split = elem.replace("-"," ").replace("s"," s").replace("h"," h").replace("m"," m").replace(":00"," min ").replace("00:"," ").replace("~"," ").replace("_"," ").replace("+"," ").replace("<"," ").replace(">"," ").split(" ")
    num = np.mean([float(s) for s in split if s.replace(".", "",1).isdigit()])
    if "minutes" in lower or "minute" in lower or "min" in lower or "mins" in lower or "m " in lower:
    #if "m " in split:
        t = 60
    elif "hours" in lower or "hour" in lower or "hrs" in lower or "hr" in lower or "h " in lower:
    #elif "h " in split:
        t = 3600
    #times.append(num*t)
    if np.isnan(num)==False:
        return num*t
    if np.isnan(num)==True:
        return t

In [177]:
#twice as fast as my code :(
def infer_duration_in_seconds(text):
    # try different regexps to extract the total seconds
    metric_text = ["hour","minute","second","sec","segundo","min","hr"," m"," h"," s"]
    metric_seconds = [3600,60,1,1,1,60,3600,60,3600,1]
    text_replaced = text.replace(":"," min ").replace("+","").replace("-"," ").lower() #check for colon
    
    #digit and metric together
    for metric,mult in zip(metric_text,metric_seconds):
        regex = "\s*(\d+)\+?\s*{}s?".format(metric)
        res = re.findall(regex,text_replaced)
        if len(res)>0:
            return int(float(res[0]) * mult)
        
    #only metric
    for metric,mult in zip(metric_text,metric_seconds):
        metric_regex = "\s*\+?\s*{}\s?".format(metric)
        res = re.findall(metric_regex,text_replaced)
        if len(res)>0:
            return mult
    
    #only digit (seconds)
    dig_regex = "\s*(\d+)\+?\s*?"
    res = re.findall(dig_regex,text_replaced)
    if len(res)>0:
        return int(float(res[0]) * 1)
    
    return None

In [178]:
t = time.time()
df['duration_seconds'] = df['duration'].map(lambda x: infer_duration_in_seconds(x))
print time.time() - t

4.14482498169


In [179]:
df.isnull().sum()

city                   0
title                  0
shape                  0
duration               0
state                  0
link                   0
date                   0
date_datetime        242
duration_seconds    6306
geo                    0
dtype: int64

###Link col

In [202]:
df['link_no_ext'] = df['link'].map(lambda x: x[:-5])

###Geocode

In [55]:
#read all geocode files (where I geocoded the addresses not initially coded) and appended to one df
files = ["1000_2000","2000_3000","3000_4000","4000_5000","5000_6000",\
         "6000_7000","7000_8000","8000_9000","9000_10000","10000_12000"]

#initialize df
geo_df_full = pd.read_csv("0_1000_geocoded.csv",header=0)
del geo_df_full['Unnamed: 0']
geo_df_full.drop_duplicates(subset=['FullLocation'],inplace=True)
geo_df_full = geo_df_full.reset_index(drop=True)

#iterate through other geocode files
for f in files:
    geo_df = pd.read_csv("{}_geocoded.csv".format(f),header=False)
    del geo_df['Unnamed: 0']
    geo_df = geo_df.reset_index(drop=True)

    geo_df_full= geo_df_full.append(geo_df,ignore_index=True)

geo_df_full.drop_duplicates(subset=['FullLocation'],inplace=True)
del geo_df_full['Unnamed: 0.1']
del geo_df_full['FullLocation']

#filter out 0's for lat/long
geo_df_full = geo_df_full[geo_df_full['lon']!=0.0]
geo_df_full = geo_df_full[geo_df_full['lat']!=0.0]

#force column names to match previous df
geo_df_full.columns = ['city','state','long','lat']

In [56]:
#full df
df = pd.read_csv("ufo_nosummary.csv")
del df["Unnamed: 0"]

#unique geocodes
df_geocodes = pd.read_csv("geocodes_8973.csv")
del df_geocodes['Unnamed: 0']
df_geocodes.drop_duplicates(subset=['city','state'],inplace=True)
df_geocodes = df_geocodes.reset_index(drop=True)

#split lat/long
df_test = pd.DataFrame(df_geocodes.latlong.str.split(',',1).tolist(),
                                   columns = ['lat','long'])
df_test['lat'] = df_test['lat'].map(lambda x: x[1:])
df_test['long'] = df_test['long'].map(lambda x: x[:-1])
del df_geocodes['latlong']

#unique geocodes with lat/long separated
df_5359_unique = pd.concat([df_geocodes,df_test],axis=1)
del df_5359_unique['link']

#merge with new geocodes (defined above)
all_the_geocodes = pd.merge(df_5359_unique,geo_df_full,how='outer')

#full df merged with unique geocodes, geocodes 56476!
df_geocoded = pd.merge(df,all_the_geocodes,how='inner',on=['city','state'])
del df_geocoded['geo']
del df_geocoded['title']
df_geocoded = df_geocoded[df_geocoded['date_datetime'].isnull()==False]

#full df merged with unique geocodes, geocodes includes NaNs
df_geocoded_full = pd.merge(df,all_the_geocodes,how='outer',on=['city','state'])
del df_geocoded_full['geo']
del df_geocoded_full['title']
df_geocoded_full = df_geocoded_full[df_geocoded_full['date_datetime'].isnull()==False]

In [57]:
df_geocoded['pre2015'] = df_geocoded['date_datetime'].map(lambda x: parser.parse(x).year < 2016)

In [61]:
df_geocoded.pre2015.value_counts()

True     82604
False     1287
dtype: int64

In [67]:
len(df_geocoded)

83891

###Time of day

In [32]:
parser.parse(df_geocoded['date_datetime'][987]).hour

20

In [58]:
df_geocoded['daylight'] = df_geocoded['date_datetime'].map(lambda x: parser.parse(x).hour>9 and parser.parse(x).hour<18)

In [59]:
df_geocoded['daylight'].value_counts()

False    71670
True     12221
dtype: int64

###USA

In [64]:
df_geocoded['USA'] = df_geocoded['state'].map(lambda x: True if x in states else False)

In [65]:
df_geocoded['USA'].value_counts()

True     78174
False     5717
dtype: int64

###See df

In [68]:
df_geocoded.to_csv("83891_locations.csv",encoding='utf-8')

In [84]:
#full
len(df_56476_full)

99477

In [75]:
#geocoded
len(df_56476)

56476

In [115]:
#not geocoded
#where both city and state are complete
len(not_yet_geocoded)

19120

In [99]:
city_states_notgeo = df_56476_full[df_56476_full['lat'].isnull()][['city','state']].drop_duplicates()
len(city_states_notgeo)

23155

In [106]:
not_yet_geocoded = city_states_notgeo[city_states_notgeo['city'].isnull()==False].reset_index(drop=True)
not_yet_geocoded = not_yet_geocoded[not_yet_geocoded['state'].isnull()==False].reset_index(drop=True)

In [114]:
not_yet_geocoded.to_csv("19120_not_geocoded.csv",encoding='utf-8')

###States dictionary

In [62]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}